In [1]:
!pip install flask sounddevice numpy
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers
!pip install diffusers
!pip install Pillow


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu121/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from flask import Flask, render_template, request
import sounddevice as sd
import numpy as np
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from diffusers import DiffusionPipeline
from PIL import Image
import io

app = Flask(__name__)

model_id = "distil-whisper/distil-small.en"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

voice_to_text_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
voice_to_text_model.to(device)

voice_processor = AutoProcessor.from_pretrained(model_id)

voice_to_text_pipe = pipeline(
    "automatic-speech-recognition",
    model=voice_to_text_model,
    tokenizer=voice_processor.tokenizer,
    feature_extractor=voice_processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)


pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
)
pipe.to("cuda")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process_voice', methods=['POST'])
def process_voice():
    if request.method == 'POST':

        audio_sample = record_voice(duration=5)


        result = voice_to_text_pipe(audio_sample)
        v_to_text = result['text']
        print(v_to_text.strip())

        prompt =v_to_text

        images = pipe(prompt=prompt).images[0]
        
        image_name = f"{v_to_text}.jpg"
        image_path = f"static/{image_name}"
        images.save(image_path)
        images

        return render_template('index.html', v_to_text=v_to_text)


def record_voice(duration=5, sample_rate=16000):
    print("Recording...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.int16)
    sd.wait()
    print("Recording done.")
    # print(audio_data.flatten().astype(np.float32) / np.iinfo(np.int16).max)
    return audio_data.flatten().astype(np.float32) / np.iinfo(np.int16).max

if __name__ == '__main__':
    app.run( port=3009, debug=False,use_reloader=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3009
Press CTRL+C to quit
127.0.0.1 - - [08/Aug/2024 21:10:48] "GET / HTTP/1.1" 200 -


Recording...
Recording done.
Rocket Bike


  0%|          | 0/50 [00:00<?, ?it/s]

127.0.0.1 - - [08/Aug/2024 21:25:28] "POST /process_voice HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 21:25:28] "GET /static/%20Rocket%20Bike.jpg HTTP/1.1" 200 -


In [ ]:
# from flask import Flask, render_template, request
# import sounddevice as sd
# import numpy as np
# import torch
# from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from diffusers import DiffusionPipeline
# from PIL import Image
# import io

# app = Flask(__name__)

# model_id = "distil-whisper/distil-small.en"
# device = "cuda" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# voice_to_text_model = AutoModelForSpeechSeq2Seq.from_pretrained(
#     model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
# )
# voice_to_text_model.to(device)

# voice_processor = AutoProcessor.from_pretrained(model_id)

# voice_to_text_pipe = pipeline(
#     "automatic-speech-recognition",
#     model=voice_to_text_model,
#     tokenizer=voice_processor.tokenizer,
#     feature_extractor=voice_processor.feature_extractor,
#     max_new_tokens=128,
#     torch_dtype=torch_dtype,
#     device=device,
# )


# pipe = DiffusionPipeline.from_pretrained(
#     "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
# )
# pipe.to("cuda")

# @app.route('/')
# def index():
#     return render_template('index.html')

# @app.route('/process_voice', methods=['POST'])
# def process_voice():
#     if request.method == 'POST':

#         audio_sample = record_voice(duration=5)


#         result = voice_to_text_pipe(audio_sample)
#         v_to_text = result['text']
#         print(v_to_text)

#         prompt =v_to_text

#         images = pipe(prompt=prompt).images[0]
        
#         image_name = f"{v_to_text}.jpg"
#         image_path = f"static/{image_name}"
#         images.save(image_path)
#         images

#         return render_template('index.html', v_to_text=v_to_text)


# def record_voice(duration=5, sample_rate=16000):
#     print("Recording...")
#     audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.int16)
#     sd.wait()
#     print("Recording done.")
#     # print(audio_data.flatten().astype(np.float32) / np.iinfo(np.int16).max)
#     return audio_data.flatten().astype(np.float32) / np.iinfo(np.int16).max

# if __name__ == '__main__':
#     app.run( port=3009, debug=False,use_reloader=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3009
Press CTRL+C to quit
127.0.0.1 - - [08/Aug/2024 08:08:19] "GET / HTTP/1.1" 200 -


Recording...
Recording done.
 Dog have feathers on wings.


  0%|          | 0/50 [00:00<?, ?it/s]

[2024-08-08 08:21:01,837] ERROR in app: Exception on /process_voice [POST]
Traceback (most recent call last):
  File "e:\voice to image model\.venv\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\voice to image model\.venv\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\voice to image model\.venv\Lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\voice to image model\.venv\Lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Asus\AppData\Local\Temp\ipykernel_27260\1731831121.py", line 56, in process_voic

In [ ]:
torch.cuda.is_available()

True